<a href="https://colab.research.google.com/github/kazeidk/GDrive_Turbo_Copy/blob/main/GDrive_Turbo_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 GDrive Turbo Copy

## ✨ Tính năng:
- ⚡ **Tối ưu tốc độ** - Giảm overhead tối đa
- 🎯 **Exact match** - Kiểm tra file chính xác
- 🔄 **Auto-resume** - Tự tiếp tục khi timeout
- 🔁 **Auto-retry** - Tự retry khi rate limit
- 📊 **Progress bar** - Hiển thị tiến độ
- 🔍 **Filter** - Lọc theo đuôi file

---

## 📖 Hướng dẫn:
1. Chạy cell **Cài đặt**
2. Chạy cell **Nhập thông tin**
3. Chạy cell **Run**

## 🔥 Recommend Colab Pro+:
- GPU: **A100** | RAM cao: **BẬT**

In [ ]:
#@title 1️⃣ Cài đặt thư viện
!pip install tqdm -q
print("✅ Đã cài đặt xong!")

In [ ]:
#@title 2️⃣ Nhập thông tin
from ipywidgets import widgets, Layout
from IPython.display import display, HTML

style = {'description_width': '150px'}
layout = Layout(width='550px')

dest_text = widgets.Text(description='📁 Folder đích:', placeholder='Link folder của bạn', style=style, layout=layout)
source_text = widgets.Text(description='📂 Folder nguồn:', placeholder='Link folder cần copy', style=style, layout=layout)
exclude_str_text = widgets.Text(description='🚫 Bỏ qua chứa:', value='', placeholder='.tmp, backup', style=style, layout=layout)
include_ext_text = widgets.Text(description='✅ Chỉ đuôi:', value='', placeholder='.pdf, .mp4', style=style, layout=layout)
exclude_ext_text = widgets.Text(description='🚫 Bỏ đuôi:', value='', placeholder='.tmp, .log', style=style, layout=layout)
skip_exists_cb = widgets.Checkbox(description='⏭️ Bỏ qua file đã có (chậm hơn)', value=True, layout=Layout(width='300px'))
dry_run_cb = widgets.Checkbox(description='🔍 Dry-run (chỉ xem)', value=False, layout=Layout(width='250px'))

display(HTML('<b>📌 Chính:</b>'))
display(dest_text, source_text)
display(HTML('<b>🔍 Lọc:</b>'))
display(exclude_str_text, include_ext_text, exclude_ext_text)
display(HTML('<b>🎛️ Khác:</b>'))
display(skip_exists_cb, dry_run_cb)
print('✅ Nhập xong chạy cell tiếp!')

In [ ]:
#@title 3️⃣ Run - GDrive Turbo Copy (MAX SPEED)
import os, time, re, json
from datetime import datetime
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.colab import auth
from tqdm.notebook import tqdm

class GDriveCopy:
    def __init__(self):
        self.total_mb = 0
        self.copied = 0
        self.skipped = 0
        self.errors = 0
        self.dry_run = False
        self.skip_exists = True
        self.exclude_str = []
        self.include_ext = []
        self.exclude_ext = []
        self.done_keys = set()
        self.checkpoint = '/content/checkpoint.json'
        self.svc = None

    def log(self, msg):
        print(f'[{datetime.now().strftime("%H:%M:%S")}] {msg}')

    def load_cp(self):
        try:
            if os.path.exists(self.checkpoint):
                d = json.load(open(self.checkpoint))
                self.done_keys = set(d.get('k', []))
                self.total_mb = d.get('m', 0)
                self.copied = d.get('c', 0)
                self.skipped = d.get('s', 0)
                if self.done_keys:
                    self.log(f'✅ Resume: {len(self.done_keys)} files')
        except: pass

    def save_cp(self):
        try:
            json.dump({'k': list(self.done_keys), 'm': self.total_mb, 'c': self.copied, 's': self.skipped}, open(self.checkpoint, 'w'))
        except: pass

    def auth(self):
        self.log('🔐 Xác thực...')
        auth.authenticate_user()
        self.svc = build('drive', 'v3')
        self.log('✅ OK')

    def ok_ext(self, name):
        if not name: return True
        ext = os.path.splitext(name)[1].lower()
        if self.include_ext: return ext in self.include_ext
        if self.exclude_ext: return ext not in self.exclude_ext
        return True

    def call(self, fn):
        for i in range(3):
            try: return fn()
            except HttpError as e:
                if e.resp.status == 404: return None
                if e.resp.status in [403, 429, 500, 502, 503]:
                    time.sleep(min(1 * (2**i), 16))  # 1s, 2s, 4s - nhanh hơn
                else: raise
            except:
                if i < 2: time.sleep(1)
                else: raise
        return None

    def ls(self, fid):
        items, token = [], None
        q = f"'{fid}' in parents and trashed=false"
        for s in self.exclude_str: q += f" and not name contains '{s}'"
        while True:
            r = self.call(lambda: self.svc.files().list(
                q=q, pageToken=token, pageSize=1000,
                fields='files(id,name,mimeType,size),nextPageToken',
                supportsAllDrives=True, includeItemsFromAllDrives=True
            ).execute())
            if not r: break
            for f in r.get('files', []):
                m = f.get('mimeType', '')
                if m == 'application/vnd.google-apps.folder' or (self.ok_ext(f.get('name')) and not m.startswith('application/vnd.google-apps.')):
                    items.append(f)
            token = r.get('nextPageToken')
            if not token: break
        return items

    def exists(self, did, name):
        if not self.skip_exists: return None
        try:
            esc = name.replace("'", "\\'")
            r = self.call(lambda: self.svc.files().list(
                q=f"'{did}' in parents and name='{esc}' and trashed=false",
                fields='files(id)', supportsAllDrives=True, pageSize=1
            ).execute())
            return r['files'][0]['id'] if r and r.get('files') else None
        except: return None

    def cp(self, did, src):
        fid, name = src['id'], src.get('name', 'unnamed')
        # Ký tự đặc biệt
        for c in '\\/:*?"<>|': name = name.replace(c, '_')
        name = name[:200].strip() or 'unnamed'
        
        size = int(src.get('size', 0))
        mb = size / (1024*1024)
        key = f"{did}:{fid}"

        # Đã copy (checkpoint)
        if key in self.done_keys:
            self.skipped += 1
            return

        # Dry-run
        if self.dry_run:
            self.log(f'[DRY] {name} ({mb:.1f}MB)')
            self.total_mb += mb
            return

        # Check exists (tùy chọn - bỏ để nhanh hơn)
        if self.skip_exists and self.exists(did, name):
            self.skipped += 1
            self.done_keys.add(key)
            return

        # COPY - phần quan trọng nhất
        try:
            t0 = time.time()
            r = self.call(lambda: self.svc.files().copy(
                body={'parents': [did], 'name': name},
                fileId=fid, supportsAllDrives=True
            ).execute())
            if r:
                spd = mb / max(time.time()-t0, 0.1)
                self.total_mb += mb
                self.copied += 1
                self.done_keys.add(key)
                self.log(f'✅ {name[:60]} ({mb:.1f}MB, {spd:.0f}MB/s)')
        except Exception as e:
            self.errors += 1
            self.log(f'❌ {name[:50]}: {str(e)[:40]}')

    def mkdir(self, did, name):
        for c in '\\/:*?"<>|': name = name.replace(c, '_')
        name = name[:200].strip() or 'unnamed'
        
        # Check folder exists
        eid = self.exists(did, name)
        if eid: return eid
        if self.dry_run: return f'dry_{name}'
        
        try:
            r = self.call(lambda: self.svc.files().create(
                body={'name': name, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [did]},
                fields='id', supportsAllDrives=True
            ).execute())
            if r:
                self.log(f'📁 {name}')
                return r['id']
        except Exception as e:
            self.log(f'❌ mkdir: {e}')
        return None

    def copy_dir(self, did, sid, depth=0):
        items = self.ls(sid)
        if not items: return

        folders = [f for f in items if f.get('mimeType') == 'application/vnd.google-apps.folder']
        files = [f for f in items if f.get('mimeType') != 'application/vnd.google-apps.folder']

        self.log(f'{"  "*depth}📊 {len(files)} files, {len(folders)} folders')

        # Copy files tuần tự - TỐI ƯU TỐC ĐỘ
        if files:
            for i, f in enumerate(tqdm(files, desc='Copying', leave=False)):
                self.cp(did, f)
                # Checkpoint mỗi 50 files (giảm I/O)
                if (i+1) % 50 == 0: self.save_cp()
            self.save_cp()  # Save cuối folder

        # Subfolders
        for folder in folders:
            fname = folder.get('name', 'unnamed')
            self.log(f'{"  "*depth}📁 {fname}')
            sub = self.mkdir(did, fname)
            if sub and not sub.startswith('dry_'):
                self.copy_dir(sub, folder['id'], depth+1)

    def get_id(self, url):
        m = re.search(r'[-\w]{25,}', url or '')
        return m.group(0) if m else None

    def run(self, dest, src):
        t0 = time.time()
        self.log('='*50)
        self.log('🚀 GDRIVE TURBO COPY - MAX SPEED')
        self.log('='*50)

        self.auth()
        self.load_cp()

        did, sid = self.get_id(dest), self.get_id(src)
        if not did or not sid:
            self.log('❌ URL không hợp lệ!')
            return

        try:
            si = self.svc.files().get(fileId=sid, supportsAllDrives=True).execute()
            di = self.svc.files().get(fileId=did, supportsAllDrives=True).execute()
        except Exception as e:
            self.log(f'❌ Không truy cập được: {e}')
            return

        self.log(f'📂 Nguồn: {si.get("name")}')
        self.log(f'📂 Đích: {di.get("name")}')
        self.log(f'⚙️ Skip exists: {self.skip_exists}')
        self.log('-'*50)

        new_dest = self.mkdir(did, si.get('name', 'Copy'))
        if not new_dest:
            self.log('❌ Không tạo được folder!')
            return

        self.copy_dir(new_dest, sid)
        self.save_cp()

        elapsed = time.time() - t0
        gb = self.total_mb / 1024
        spd = self.total_mb / max(elapsed, 1)

        self.log('='*50)
        self.log(f'✅ {self.copied} | ⏭️ {self.skipped} | ❌ {self.errors}')
        self.log(f'💾 {gb:.2f}GB | ⏱️ {int(elapsed)}s | 🚀 {spd:.1f}MB/s')
        self.log('='*50)
        self.log('🎉 XONG!' if self.errors == 0 else f'⚠️ {self.errors} lỗi')

# === RUN ===
print("🔧 Khởi tạo...")
dl = GDriveCopy()
dl.exclude_str = [s.strip() for s in exclude_str_text.value.split(',') if s.strip()]
dl.include_ext = [('.' + s.strip().lstrip('.').lower()) for s in include_ext_text.value.split(',') if s.strip()]
dl.exclude_ext = [('.' + s.strip().lstrip('.').lower()) for s in exclude_ext_text.value.split(',') if s.strip()]
dl.skip_exists = skip_exists_cb.value
dl.dry_run = dry_run_cb.value

if not dest_text.value.strip(): print("❌ Nhập folder đích!")
elif not source_text.value.strip(): print("❌ Nhập folder nguồn!")
else: dl.run(dest_text.value.strip(), source_text.value.strip())

In [ ]:
#@title 4️⃣ Xóa checkpoint (chạy lại từ đầu)
import os
for f in ['/content/checkpoint.json']:
    if os.path.exists(f): os.remove(f); print(f'✅ Xóa {f}')
    else: print(f'⏭️ Không có {f}')
print('🔄 OK!')